In [53]:
from __future__ import absolute_import
#from __future__ import absolute_import means that if you import string,
#Python will always look for a top-level string module, rather than current_package.string.
from __future__ import print_function
#This ensures compatibility in Python versions 3.x and 2.x.
#In 2.x, print “x” was allowed, 3.x accepts only print(“x”)
import tensorflow as tf
import numpy as np
import pandas as pd

In [54]:
data_file_name = "bbdata.txt"
# name of the file which contains our dataset
column_names = "id,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_epith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class"
# column names in a comma seperated way

#we are opening the file which contain dataset and putting name of columns in first row
with open(data_file_name, "r+") as f:
    #putting column names on the first row.
    datainfile = f.read()
    f.seek(0, 0)
    f.write(column_names.rstrip('\r\n') + '\n' + datainfile)

In [55]:
# now we are reading the above csv file using pandas dataframe
df = pd.read_csv(data_file_name)

In [56]:
df.replace('?', np.nan, inplace = True)
# the above code will replace all missing values with nan( standard notation for missing values)
df.dropna(inplace = True)
# here we are dropping rows which contains Nan values as those will be of no use to us.

In [57]:
df.drop(['id'], axis = 1, inplace = True)
# here we are dropping column name id 

In [62]:
df['class'].replace(2, 0, inplace = True)
df['class'].replace(4, 1, inplace = True)
# here we are replacing 2 with 0 and 4 with 1
# 0 and 1 stands for cancer typer M = malignant, B = benign

In [64]:
df.to_csv("combined_data.csv", index = False)
#saving the resultant dataset into a csv file for later use

In [66]:
print(df["class"])
# class values 

0      0
1      0
2      0
3      0
4      0
5      1
6      0
7      0
8      0
9      0
10     0
11     0
12     1
13     0
14     1
15     1
16     0
17     0
18     1
19     0
20     1
21     1
22     0
24     0
25     1
26     0
27     0
28     0
29     0
30     0
      ..
669    1
670    1
671    0
672    0
673    0
674    0
675    0
676    0
677    0
678    0
679    0
680    1
681    1
682    0
683    0
684    0
685    0
686    0
687    0
688    0
689    0
690    0
691    1
692    0
693    0
694    0
695    0
696    1
697    1
698    1
Name: class, Length: 683, dtype: int64


In [204]:
import numpy as np

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=2)]
print (feature_columns)

dataset = pd.read_csv('combined_data.csv',header=None)
# Y=dataset['class']
# dataset.drop("class",1,inplace=True)
x=dataset
x.drop(x.index[0], inplace=True)
#x.loc[0] = [x.shape[0],x.shape[1],"benign","malignant",np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

indices = np.random.permutation(x.shape[0])
k=.8*x.shape[0]
k=int(k)
training_idx, test_idx = indices[:k], indices[k:]
#print(a)
training, test = x.loc[training_idx,:], x.loc[test_idx,:]
print (training.shape)
print (test.shape)
training.loc[indices[0]] = [training.shape[0]-1,training.shape[1]-1,"benign","malignant",np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
test.loc[indices[k]] = [test.shape[0]-1,test.shape[1]-1,"benign","malignant",np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
#training.reset_index(drop=True, inplace=True)
print(training.head())
print(test.head())
test.to_csv("test_new.csv", header=False,index = False)
training.to_csv("train_new.csv",header=False, index = False)



[_RealValuedColumn(column_name='', dimension=2, default_value=None, dtype=tf.float32, normalizer=None)]
(546, 10)
(137, 10)
       0  1       2          3    4    5    6    7    8    9
418  545  9  benign  malignant  NaN  NaN  NaN  NaN  NaN  NaN
87     4  1       1          1    2    1    3    1    1    0
390    3  1       1          1    2    4    1    1    1    0
541    4  3       1          1    2    1    4    8    1    0
379    3  1       1          1    2    1    2    1    1    0
       0  1       2          3    4    5    6    7    8    9
537  136  9  benign  malignant  NaN  NaN  NaN  NaN  NaN  NaN
653    5  7       4          1    6    1    7   10    3    1
199    1  1       1          1    2    1    3    1    1    0
450    4  1       1          1    2    1    1    1    1    0
162   10  8      10         10    6    1    3    1   10    1


In [210]:
training_data = "train_bc.csv"
testing_data = "test_new.csv"

# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=training_data,
                                                       target_dtype=np.int,
                                                       features_dtype=np.float32,
                                                       target_column=-1)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=testing_data,
                                                   target_dtype=np.int,
                                                   features_dtype=np.float32,
                                                   target_column=-1)

In [211]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=2,
                                            model_dir="/tmp/cancer_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021911CCD160>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/cancer_model'}


In [212]:
classifier.fit(x = training_set.data,
               y = training_set.target,
               steps = 2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/cancer_model\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4000 into /tmp/cancer_model\model.ckpt.
INFO:tensorflow:loss = 0.022666171, step = 4001
INFO:tensorflow:global_step/sec: 255.798
INFO:tensorflow:loss = 0.022182217, step = 4101 (0.394 sec)
INFO:tensorflow:global_step/sec: 319.059
INFO:tensorflow:loss = 0.02162641, step = 4201 (0.277 sec)
INFO:tensorflow:global_step/sec: 367.634
INFO:tensorflow:loss = 0.02116517, step = 4301 (0.271 sec)
INFO:tensorflow:global_step/sec: 353.412
INFO:tensorflow:loss = 0.020651203, step = 4401 (0.284 sec)
INFO:tensorflow:global_step/sec: 335.623
INFO:tensorflow:loss = 0.02024334, step = 4501 (0.299 sec)
INFO:tensorflow:global_step/sec: 336.754
INFO:tensorflow:loss = 0.01973633, step = 4601 (0.296 sec)
INFO:tensorflow:global_step/sec

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x0000021911CCD6D8>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=2, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000002190CA559D8>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [213]:
accuracy_score = classifier.evaluate(x = test_set.data,
                                     y = test_set.target)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Starting evaluation at 2019-03-31-18:07:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/cancer_model\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-31-18:07:12
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.99264705, accur

In [214]:
# Classify two new cancer tumor samples.
def new_samples():
  return np.array([[5, 10, 8, 4, 7, 4, 8, 11, 2],
                   [5, 1, 1, 1, 1, 1, 1, 1, 2]], dtype=np.float32)

predictions = list(classifier.predict(input_fn=new_samples))

print(
      "New Samples, Class Predictions:    {}\n"
      .format(predictions))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/cancer_model\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
New Samples, Class Predictions:    [1, 0]

